In [1]:
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
from torchvision.models import resnet50, ResNet50_Weights, ResNet18_Weights, resnet18, EfficientNet_B0_Weights, efficientnet_b0
import random
import torch
class SiameseResNet(nn.Module):
    def __init__(self):
        super(SiameseResNet, self).__init__()


        self.resnet18  = models.resnet18()
        ct = 0
        for child in self.resnet18.children():
            ct += 1
            if ct < 9:
                for param in child.parameters():
                    param.requires_grad = False

        blocks = list(self.resnet18.layer4.children())[-1]

        for param in blocks.parameters():
                param.requires_grad = True
        # Remove the final layer to get embeddings
        self.features = nn.Sequential(*list(self.resnet18.children())[:-1])

        self.dense = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )



    def forward_one(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.dense(x)
        return x

    def forward(self, anchor, positive, negative):
        anchor_embedding = self.forward_one(anchor)
        positive_embedding = self.forward_one(positive)
        negative_embedding = self.forward_one(negative)
        return anchor_embedding, positive_embedding, negative_embedding

In [3]:
def check_frozen(model):
    for name, param in model.named_parameters():
        if not param.requires_grad:
            print(f"Module {name} is frozen")
        else:
            print(f"Module {name} is NOT frozen")

# example usage:
model = SiameseResNet()
check_frozen(model)

Module resnet18.conv1.weight is frozen
Module resnet18.bn1.weight is frozen
Module resnet18.bn1.bias is frozen
Module resnet18.layer1.0.conv1.weight is frozen
Module resnet18.layer1.0.bn1.weight is frozen
Module resnet18.layer1.0.bn1.bias is frozen
Module resnet18.layer1.0.conv2.weight is frozen
Module resnet18.layer1.0.bn2.weight is frozen
Module resnet18.layer1.0.bn2.bias is frozen
Module resnet18.layer1.1.conv1.weight is frozen
Module resnet18.layer1.1.bn1.weight is frozen
Module resnet18.layer1.1.bn1.bias is frozen
Module resnet18.layer1.1.conv2.weight is frozen
Module resnet18.layer1.1.bn2.weight is frozen
Module resnet18.layer1.1.bn2.bias is frozen
Module resnet18.layer2.0.conv1.weight is frozen
Module resnet18.layer2.0.bn1.weight is frozen
Module resnet18.layer2.0.bn1.bias is frozen
Module resnet18.layer2.0.conv2.weight is frozen
Module resnet18.layer2.0.bn2.weight is frozen
Module resnet18.layer2.0.bn2.bias is frozen
Module resnet18.layer2.0.downsample.0.weight is frozen
Module

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
pair_train = pd.read_csv('pair.csv')
anchors = []
positives = []
negatives = []
for index, row in pair_train.iterrows():
    for negative in row[2:]:
        anchors.append(row[0])
        positives.append(row[1])
        negatives.append(negative)


# Combine the lists into a single list of tuples
data = list(zip(anchors, positives, negatives))

# Split the combined list into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Separate the training and validation sets into anchors, positives, and negatives
train_anchors, train_positives, train_negatives = zip(*train_data)
val_anchors, val_positives, val_negatives = zip(*val_data)

In [6]:

class ImageTripletDataset(Dataset):
    def __init__(self, anchors, positives, negatives, tensor_dict_left, tensor_dict_right, all_keys_right):
        self.anchors = anchors
        self.positives = positives
        self.negatives = negatives
        self.tensor_dict_left = tensor_dict_left
        self.tensor_dict_right = tensor_dict_right
        self.all_keys_right = all_keys_right


    def __getitem__(self, idx):
        anchor = self.tensor_dict_left[self.anchors[idx]]
        positive = self.tensor_dict_right[self.positives[idx]]

        if random.random() < 0.5:
            possible_keys = self.all_keys_right.copy()
            possible_keys.remove(self.positives[idx])
            possible_keys.remove(self.negatives[idx])

            negative = self.tensor_dict_right[random.choice(all_keys_right)]

        else:
            negative = self.tensor_dict_right[self.negatives[idx]]

        return anchor, positive, negative

    def __len__(self):
        return len(self.anchors)


In [7]:
def load_tensors_from_folder(tensor_folder):
    tensor_files = [f for f in os.listdir(tensor_folder) if f.endswith('.pt')]
    tensor_dict = {}

    for tensor_file in tensor_files:
        tensor_path = os.path.join(tensor_folder, tensor_file)
        tensor_name = os.path.splitext(tensor_file)[0]  # Remove the .pt extension
        tensor_dict[tensor_name] = torch.load(tensor_path)

    return tensor_dict

# Usage example
tensor_folder_left = '/content/tensor/train_tensor/left'
tensor_dict_left = load_tensors_from_folder(tensor_folder_left)
tensor_folder_right = '/content/tensor/train_tensor/right'
tensor_dict_right = load_tensors_from_folder(tensor_folder_right)
all_keys_right =  list(tensor_dict_right.keys())

In [10]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
device = torch.device('cuda')
model = SiameseResNet().to(device)
criterion = nn.TripletMarginLoss(margin=1, p=2, eps=1e-7)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_data_loader = DataLoader(ImageTripletDataset(train_anchors, train_positives, train_negatives, tensor_dict_left, tensor_dict_right, all_keys_right), batch_size=64, shuffle=True)
val_data_loader = DataLoader(ImageTripletDataset(val_anchors, val_positives, val_negatives, tensor_dict_left, tensor_dict_right, all_keys_right), batch_size=64, shuffle=False)

In [11]:
num_epochs = 15
patience = 2
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", position=0, leave=True)

    for anchor, positive, negative in progress_bar:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        optimizer.zero_grad()
        anchor_embed, positive_embed, negative_embed = model(anchor, positive, negative)
        loss = criterion(anchor_embed, positive_embed, negative_embed)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"loss": epoch_loss / (progress_bar.n + 1)})

    avg_epoch_loss = epoch_loss / len(train_data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_epoch_loss}")

    # Validation
    model.eval()
    val_losses = []

    with torch.no_grad():
        for anchor, positive, negative in val_data_loader:
            anchor_embed, positive_embed, negative_embed = model(anchor.to(device), positive.to(device), negative.to(device))
            loss_val = criterion(anchor_embed, positive_embed, negative_embed)
            val_losses.append(loss_val.item())

    val_loss = sum(val_losses) / len(val_losses)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}")

    if val_loss  < best_loss:
        best_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'fine_tuned_model_new.pth')
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping due to no improvement in training loss.")
        break

    model.train()  # Set model back to training mode for next epoch

Epoch 1/15: 100%|██████████| 475/475 [01:20<00:00,  5.87it/s, loss=0.442]


Epoch 1/15, Average Loss: 0.4415504180130206
Epoch 1/15, Validation Loss: 0.12569783847121632


Epoch 2/15: 100%|██████████| 475/475 [01:24<00:00,  5.61it/s, loss=0.0759]


Epoch 2/15, Average Loss: 0.07593917252201783
Epoch 2/15, Validation Loss: 0.04191102348894131


Epoch 3/15: 100%|██████████| 475/475 [01:24<00:00,  5.64it/s, loss=0.0356]


...

Epoch 5/15, Average Loss: 0.01954876429156253
Epoch 5/15, Validation Loss: 0.012155964391446915


Epoch 6/15: 100%|██████████| 475/475 [01:24<00:00,  5.62it/s, loss=0.0185]


Epoch 6/15, Average Loss: 0.018505192869587947
Epoch 6/15, Validation Loss: 0.016320316731428898


Epoch 7/15: 100%|██████████| 475/475 [01:24<00:00,  5.63it/s, loss=0.0182]


Epoch 7/15, Average Loss: 0.01819820866773003
Epoch 7/15, Validation Loss: 0.01559519542365515
Early stopping due to no improvement in training loss.
